# Exercise 2.3 Contour plots (cartopy)
prepared by M.Hauser

Besides `pcolormesh` we can also do contour and filled contour plots in matplotlib. This is done with `contour` and `contourf`.

Note that most of what we show here for georeferenced plots also applies to normal `contour` and `contourf`.

## Import libraries

In [ ]:
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr

In [ ]:
import mplotutils as mpu

## Plot filled contours (`contourf`)

`contourf` works similar to `pcolormesh` and takes x, y, z as input:

In [ ]:
# create sample data
lon, lat, data = mpu.sample_data_map(90, 48)

# ====

f, ax = plt.subplots(subplot_kw=dict(projection=ccrs.PlateCarree()))
ax.coastlines()

h = ax.contourf(lon, lat, data)

There is one white stripe - the reason is that the latitude coordinates go from 0°E to 356°E (in this example) while the map is from -180°E to 180°E. The solution is to add a wrap-around point, i.e. repeat the first column at the end. This is done with the function `add_cyclic_point` which can be found in `cartopy.util`:

In [ ]:
import cartopy.util as cutil

This adds an additional column for `data` and `lon`:

In [ ]:
lon, lat, data = mpu.sample_data_map(90, 48)
DATA, LON = cutil.add_cyclic_point(data, lon)

print(f"{lon.shape=}")
print(f"{LON.shape=}")
print()
print(f"{data.shape=}")
print(f"{DATA.shape=}")

And now the map is filled:

In [ ]:
# create sample data
lon, lat, data = mpu.sample_data_map(90, 48)
DATA, LON = cutil.add_cyclic_point(data, lon)

# ====

f, ax = plt.subplots(subplot_kw=dict(projection=ccrs.PlateCarree()))
ax.coastlines()

h = ax.contourf(LON, lat, DATA)

> Note: adding a cyclic point does not need to be done for non-map contour plots.

## Load data

For the exercises we load a NetCDF with historical, and projected climatological precipitation, as well as the relative change between them, from all CMIP5 models for RCP8.5 (Taylor et al., 2012).

The data was prepared in [another notebook](../data/prepare_CMIP5_map.ipynb).

In [ ]:
file = "../data/cmip5_delta_pr_rcp85_map.nc"

# load data, omitting some unnecessary variables
pr = xr.open_dataset(file, drop_variables=["proj", "agree_sign", "pval"])

### Exercise
 * Plot the climatological precipitation amount - make sure to get rid of the white band.
 * Add a colorbar

In [ ]:
# get data
lon, lat, hist = pr.lon, pr.lat, pr.hist

# ====

f, ax = plt.subplots(subplot_kw=dict(projection=ccrs.Robinson()))

ax.coastlines()

# code here

### Solution

In [ ]:
# get data
lon, lat, hist = pr.lon, pr.lat, pr.hist
HIST, LON = cutil.add_cyclic_point(hist, lon)

# ====

f, ax = plt.subplots(subplot_kw=dict(projection=ccrs.Robinson()))

ax.coastlines()

h = ax.contourf(LON, lat, HIST, transform=ccrs.PlateCarree())
plt.colorbar(h)

## `levels`

`contourf` (and `contour`) take a `levels` keyword to manually set the levels to draw, e.g.:

```python

levels = np.arange(0, 1.1, 0.25)
levels = [-1, 0, 1]
```

Because you can directly pass the levels to the function, this is much easier than in `ax.pcolormesh`.

### Note on np.arange
`np.arange` follows the same logic as `range`: the upper bound needs to be larger than where we want it to stop, because it starts at 0. 

In [ ]:
print(list(range(10)))
print(np.arange(10))

print()

print("Wrong:")
print(repr(np.arange(0, 1, 0.5)))

print("Correct:")
print(repr(np.arange(0, 1.1, 0.5)))

We set three levels to our example data: 

In [ ]:
# create sample data
lon, lat, data = mpu.sample_data_map(90, 48)
DATA, LON = cutil.add_cyclic_point(data, lon)

levels = np.arange(-0.75, 1.1, 0.5)

# ====

f, ax = plt.subplots(subplot_kw=dict(projection=ccrs.PlateCarree()))

ax.coastlines()

h = ax.contourf(LON, lat, DATA, levels=levels, cmap="RdYlBu")
plt.colorbar(h)

## `extend`

This is probably not what we want - it clips values above/ below our `levels`. To fix this we can use the `extend` keyword:

In [ ]:
# create sample data
lon, lat, data = mpu.sample_data_map(90, 48)
DATA, LON = cutil.add_cyclic_point(data, lon)

levels = np.arange(-0.75, 1.1, 0.5)

# ====

f, ax = plt.subplots(subplot_kw=dict(projection=ccrs.PlateCarree()))

ax.coastlines()

h = ax.contourf(LON, lat, DATA, levels=levels, cmap="RdYlBu", extend="both")
plt.colorbar(h)

### Exercise

 * Create the levels `0-1000`, `1000-2000`, `2000-3000` for the precipitation plot
 * Don't forget to indicate that values > 3000 are clipped

In [ ]:
# get data
lon, lat, hist = pr.lon, pr.lat, pr.hist
HIST, LON = cutil.add_cyclic_point(hist, lon)

# levels = ...

# ====

f, ax = plt.subplots(subplot_kw=dict(projection=ccrs.Robinson()))

ax.coastlines()

h = ax.contourf(LON, lat, HIST, transform=ccrs.PlateCarree())
plt.colorbar(h)

### Solution

In [ ]:
# get data
lon, lat, hist = pr.lon, pr.lat, pr.hist
HIST, LON = cutil.add_cyclic_point(hist, lon)

levels = np.arange(0, 3100, 1000)

# ====

f, ax = plt.subplots(subplot_kw=dict(projection=ccrs.Robinson()))

ax.coastlines()

h = ax.contourf(
    LON, lat, HIST, transform=ccrs.PlateCarree(), levels=levels, extend="max"
)
plt.colorbar(h)

## xarray

Similarly to `pcolormesh` we can use `xarray` to directly create contour plots:
   
```python
ds.data.plot.contour(ax=ax, ...)

ds.data.plot.contourf(ax=ax, ...)
```

xarray
 * automatically chooses a divergent colormap if the values cross 0
 * automatically set `extend` appropriately
 * does _not_ automatically add cyclic points. We can use `mpu.cyclic_dataarray` for this.
 
#### Example

In [ ]:
sample_da = mpu.sample_dataarray(90, 48)

sample_da_cyclic = mpu.cyclic_dataarray(sample_da)

# ====

f, ax = plt.subplots(subplot_kw=dict(projection=ccrs.Robinson()))
ax.coastlines(color="0.4")

sample_da_cyclic.plot.contourf(ax=ax, transform=ccrs.PlateCarree(), add_colorbar=False)

### Exercise

 * Plot the CMIP5 precipitation data with xarray (`pr.hist`)
 * Also set the levels with steps of 1000


In [ ]:
# pr_hist_cyclic =
# levels =

# ---

f, ax = plt.subplots(subplot_kw=dict(projection=ccrs.Robinson()))

### Solution

In [ ]:
pr_hist_cyclic = mpu.cyclic_dataarray(pr.hist)
levels = np.arange(0, 3001, 1000)

# ----

f, ax = plt.subplots(subplot_kw=dict(projection=ccrs.Robinson()))
ax.coastlines()

pr_hist_cyclic.plot.contourf(ax=ax, transform=ccrs.PlateCarree(), levels=levels)

## Plot contour lines (`contour`)

`contour` plots contours that are not filled. `contour` takes the same arguments as `contourf` and additionally takes a `linewidths` and a `linestyles` parameter. If the we choose a monochrome color, negative contours are dashed, unless otherwise specified.

See the [documentation](https://matplotlib.org/stable/api/_as_gen/matplotlib.axes.Axes.contourf.html) for details.

> Below we illustrate _both_ the regular pyplot interface as well as the one from xarray.

In [ ]:
# create sample data

# numpy array
lon, lat, data = mpu.sample_data_map(90, 48)
DATA, LON = cutil.add_cyclic_point(data, lon)

# xr.DataArray
sample_da = mpu.sample_dataarray(90, 48)
sample_da_cyclic = mpu.cyclic_dataarray(sample_da)

# ====

f, axs = plt.subplots(2, 1, subplot_kw=dict(projection=ccrs.PlateCarree()))

ax = axs[0]

ax.coastlines(color="0.5")
h = ax.contour(
    LON,
    lat,
    DATA,
    levels=[-0.5, 0.75],
    colors=".1",
    transform=ccrs.PlateCarree(),
)
ax.set_title("colors='.1'; linestyles not set")

# ==

ax = axs[1]

ax.coastlines(color="0.5")
h = sample_da_cyclic.plot.contour(
    ax=ax,
    levels=[-0.5, 0.75],
    colors=".1",
    linestyles="-",
    transform=ccrs.PlateCarree(),
)
ax.set_title("colors='.1', linestyles='-'")

### Exercise
 * Add contourlines for the relative change of precipitation in the next century
 * Indicate a decrease of -25 % with brown (`"#8c510a"`)
 * Indicate an increase of 25 % with green (`"#7fbc41"`)

In [ ]:
# get data
lon, lat, pr_rel = pr.lon, pr.lat, pr.pr_rel
PR_REL, LON = cutil.add_cyclic_point(pr_rel, lon)

# levels =
colors = ["#8c510a", "#7fbc41"]

# ====

f, ax = plt.subplots(subplot_kw=dict(projection=ccrs.Robinson()))

ax.coastlines()

# h = ax.contour(...)

### Solution

In [ ]:
# get data
lon, lat, pr_rel = pr.lon, pr.lat, pr.pr_rel
PR_REL, LON = cutil.add_cyclic_point(pr_rel, lon)

levels = [-25, 25]
colors = ["#8c510a", "#7fbc41"]

# ====

f, ax = plt.subplots(subplot_kw=dict(projection=ccrs.Robinson()))

ax.coastlines()

h = ax.contour(
    LON, lat, PR_REL, transform=ccrs.PlateCarree(), levels=levels, colors=colors
)

## Bonus

* Label contour lines
* Additional xarray exercise

### contour label

The contour lines can be labeled by passing `h` to `ax.clabel`:

In [ ]:
# create sample data
lon, lat, data = mpu.sample_data_map(90, 48)
DATA, LON = cutil.add_cyclic_point(data, lon)

# ====

f, ax = plt.subplots(1, 1, subplot_kw=dict(projection=ccrs.PlateCarree()))

ax.coastlines()
h = ax.contour(LON, lat, DATA, levels=[-0.5, 0.75], transform=ccrs.PlateCarree())

cl = ax.clabel(h)

### Exercise
 * Add labels to the contourlines for the relative change of precipitation
 * Specify the number format as `fmt='%1.0f %%'` to get '25 %'

In [ ]:
# get data
pr_cyclic = mpu.cyclic_dataarray(pr)

levels = [-25, 25]
colors = ["#8c510a", "#7fbc41"]

# ====

f, ax = plt.subplots(subplot_kw=dict(projection=ccrs.Robinson()))

ax.coastlines()

h = pr_cyclic.pr_rel.plot.contour(
    ax=ax, transform=ccrs.PlateCarree(), levels=levels, colors=colors, extend="both"
)

In [ ]:
pr_cyclic.hist

### Solution

In [ ]:
# get data
pr_cyclic = mpu.cyclic_dataarray(pr)

levels = [-25, 25]
colors = ["#8c510a", "#7fbc41"]

# ====

f, ax = plt.subplots(subplot_kw=dict(projection=ccrs.Robinson()))

ax.coastlines()

h = pr_cyclic.pr_rel.plot.contour(
    ax=ax, transform=ccrs.PlateCarree(), levels=levels, colors=colors, extend="both"
)

cl = ax.clabel(h, fmt="%1.0f %%")

### Additional xarray exercise

* Plot the temperature in °Celsius
* Manually select levels

In [ ]:
# load temperature data

file = "../data/cesm_temp.nc"

cesm = xr.open_dataset(file)

In [ ]:
temp = cesm.temp - 273.15
# levels =

# ====

In [ ]:
# solution

temp = cesm.temp - 273.15

temp_cyclic = mpu.cyclic_dataarray(temp)

# ====

f, ax = plt.subplots(1, 1, subplot_kw=dict(projection=ccrs.Robinson()))

ax.coastlines()

levels = np.arange(-35, 36, 5)

temp_cyclic.plot.contourf(
    ax=ax, transform=ccrs.PlateCarree(), levels=levels, extend="both"
)